# Prepare sort-seq dataset for use in MAVE-NN

In [1]:
# Standard imports
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
%matplotlib inline

# Insert mavenn at beginning of path
import sys
path_to_mavenn_local = '../../../../'
sys.path.insert(0,path_to_mavenn_local)

#Load mavenn and check path
import mavenn
print(mavenn.__path__)

# For testing
from mavenn.src.utils import vec_data_to_mat_data

['../../../../mavenn']


In [2]:
# Shit, these sequences aren't the right length! Get full sequences from Kinney et al., 2010
# I can mess with this later; would be nice to include full sequence
len('GGCTGTTCACTTTATGCTTCCGGCTTGTATTTTGTGTGC')

39

In [3]:
# Load raw data file
raw_df = pd.read_csv(mavenn.__path__[0] +
    '/examples/datasets/sort_seq/full-wt/full-wt-sort_seq.csv',
    index_col=[0])
raw_df.head()

FileNotFoundError: [Errno 2] No such file or directory: '../../../../mavenn/examples/datasets/sort_seq/full-wt/full-wt-sort_seq.csv'

In [ ]:
# Refine contents of raw data file
sequences = raw_df['seq'].values
raw_df.columns = ['x','y','ct']
raw_df['ct'] = raw_df['ct'].astype(int)
raw_df.head()

In [ ]:
# Pivot and set training/test data
data_df = pd.pivot(raw_df, values='ct', index='x', columns='y').fillna(0).astype(int)
data_df.columns.name = None

# Get y_cols
data_df.columns = [f'ct_{x}' for x in data_df.columns]
y_cols = list(data_df.columns)

# Do all columns still sum to > 0?
print('rows summing to 0:', (data_df.values.sum(axis=1)==0).sum())
data_df.head()

In [ ]:
N = len(data_df)
training_frac=.8
np.random.seed(0)
data_df['training_set'] = (np.random.rand(N) < training_frac)
data_df.reset_index(inplace=True)
data_df.head()

In [ ]:
# Remove entries where ct is 0
ix = data_df[y_cols].sum(axis=1) > 0
print(f'Dropping {sum(~ix)} columns with 0 counts.')
data_df = data_df[ix].reset_index(drop=True)
data_df.head()

In [ ]:
# Assign to trianing and test sets
N = len(data_df)
training_frac=.8
np.random.seed(0)
r = np.random.rand(N)
test_frac = .2
val_frac = .2
ix_train = (test_frac + val_frac <= r)
ix_val = (test_frac <= r) & (r < test_frac + val_frac)
ix_test = (r < test_frac)
data_df['set'] = ''
data_df.loc[ix_train, 'set'] = 'training'
data_df.loc[ix_val, 'set'] = 'validation'
data_df.loc[ix_test, 'set'] = 'test'
assert all([len(x)>0 for x in data_df['set']])

# Shuffle data for extra safety
data_df = data_df.sample(frac=1).reset_index(drop=True)

# Order columns
data_df = data_df[['set'] + y_cols + ['x']]
data_df.head(20)

In [ ]:
# Show size of compressed dataset file
file_name = 'sortseq_data.csv.gz'
data_df.to_csv(file_name, compression='gzip', index=False)
print('df (zipped):')
!du -mh $file_name
!mv $file_name ../.

In [ ]:
# Test loading
loaded_df = mavenn.load_example_dataset('sortseq')
loaded_df.head()